LCEL makes it easy to build complex chains from basic components, and supports out of the box functionality such as streaming, parallelism, and logging.

## Basic example: prompt + model + output parser

The most basic and common use case is chaining a prompt template and a model together. To see how this works, let's create a chain that takes a topic and generates a joke:

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")
model = ChatOpenAI(model="gpt-3.5-turbo")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "ice cream"})

Notice this line of this code, where we piece together the different components into a single chain:

In [ ]:
chain = prompt | model | output_parser

The `|` symbol is similar to a unix pipe operator, creating a chain in which the output of each component is fed as input into the next component.

In this chain the user input is passed to the prompt template, then the prompt template output is passed to the model, then the model output is passed to the output parser. Let's take a look at each component individually to really understand what's going on. 

### 1. Prompt

`prompt` is a `BasePromptTemplate`, which means it takes in a dictionary of template variables and produces a `PromptValue`. A `PromptValue` is a wrapper around a completed prompt that can be passed to either an `LLM` or `ChatModel`. It can work with either language model type because it defines logic both for producing `BaseMessage`s and for producing a string.

In [4]:
prompt_value = prompt.invoke({"topic": "ice cream"})
prompt_value

ChatPromptValue(messages=[HumanMessage(content='Tell me a short joke about ice cream')])

In [6]:
prompt_value.to_messages()

[HumanMessage(content='Tell me a short joke about ice cream')]

In [7]:
prompt_value.to_string()

'Human: Tell me a short joke about ice cream'

### 2. Model

The `PromptValue` is then passed to `model`. In this case our `model` is a `ChatModel`, meaning it will output a `BaseMessage`.

In [8]:
message = model.invoke(prompt_value)
message

AIMessage(content='Why did the ice cream go to therapy? \n\nBecause it was feeling a little rocky road!')

If our `model` was an `LLM`, it would output a string.

In [9]:
from langchain.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-instruct")
llm.invoke(prompt_value)

'\n\nWhy did the ice cream go to therapy?\n\nBecause it was feeling a little soft serve.'

### 3. Output parser

And lastly we pass our `model` output to the `output_parser`, which is a `BaseOutputParser` meaning it takes either a string or a 
`BaseMessage` as input. The `StrOutputParser` specifically simple converts any input into a string.

In [10]:
output_parser.invoke(message)

'Why did the ice cream go to therapy? \n\nBecause it was feeling a little rocky road!'

## Why use LCEL

We could recreate our above chain functionality without LCEL or LangChain at all by doing something like this:

In [29]:
import openai


def manual_chain(topic: str) -> str:
    prompt_value = f"Tell me a short joke about {topic}"
    client = openai.OpenAI()
    response = client.chat.completions.create(
        model="gpt-3.5-turbo", messages=[{"role": "user", "content": prompt_value}]
    )
    return response.choices[0].message.content


manual_chain("ice cream")

"Why did the ice cream go to therapy?\nBecause it had too many toppings and couldn't find its scoop!"

If we want to stream results instead, we'll need to change oour function:

In [31]:
def manual_chain_stream(topic: str) -> str:
    prompt_value = f"Tell me a short joke about {topic}"
    client = openai.OpenAI()
    stream = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt_value}],
        stream=True,
    )
    for response in stream:
        yield response.choices[0].delta.content


for chunk in manual_chain_stream("ice cream"):
    print(chunk, end="", flush=True)

Why do ice cream cones never listen?

Because they're just too vanilla!None

If we want to run on a batch of inputs in parallel, we'll again need a new function:

In [34]:
from concurrent.futures import ThreadPoolExecutor


def manual_chain_batch(topics: list) -> list:
    with ThreadPoolExecutor(max_workers=5) as executor:
        return list(executor.map(manual_chain, topics))


manual_chain_batch(["ice cream", "spaghetti", "dumplings"])

['Why did the girl bring a ladder to the ice cream shop? \n\nBecause she wanted to reach the sundae on the top shelf!',
 'Why did the spaghetti go to the party?\n\nBecause it wanted to meatball!',
 'Why did the dumpling go to the gym? \n\nBecause it wanted to become a "well-rounded" snack!']

If you needed an asynchronous version:

In [47]:
async def manual_chain_async(topic: str) -> str:
    prompt_value = f"Tell me a short joke about {topic}"
    client = openai.AsyncOpenAI()
    response = await client.chat.completions.create(
        model="gpt-3.5-turbo", messages=[{"role": "user", "content": prompt_value}]
    )
    return response.choices[0].message.content

If we want to use a completion endpoint instead of a chat endpoint: 

In [39]:
def manual_chain_completion(topic: str) -> str:
    prompt_value = f"Tell me a short joke about {topic}"
    client = openai.OpenAI()
    response = client.completions.create(
        model="gpt-3.5-turbo-instruct",
        prompt=prompt_value,
    )
    return response.choices[0].text


print(manual_chain_completion("ice cream"))



Why did the ice cream go to therapy? 
Because it was feeling


If we want to use Anthropic instead of OpenAI: 

In [43]:
import anthropic


def manual_chain_anthropic(topic: str) -> str:
    prompt_value = f"Human:\n\nTell me a short joke about {topic}\n\nAssistant:"
    client = anthropic.Anthropic()
    response = client.completions.create(
        model="claude-2",
        prompt=prompt_value,
        max_tokens_to_sample=256,
    )
    return response.completion


print(manual_chain_anthropic("ice cream"))

 Here's a silly short joke about ice cream:

What kind of shoes do frogs wear? Open-toad sandals!


If we want to log our intermediate results (we'll `print` here for illustrative purposes):

In [46]:
def manual_chain_anthropic_logging(topic: str) -> str:
    print(f"Input: {topic}")
    prompt_value = f"Human:\n\nTell me a short joke about {topic}\n\nAssistant:"
    print(f"Formatted prompt: {prompt_value}")
    client = anthropic.Anthropic()
    response = client.completions.create(
        model="claude-2",
        prompt=prompt_value,
        max_tokens_to_sample=256,
    )
    print(f"Output: {response.completion}")
    return response.completion


print(manual_chain_anthropic_logging("ice cream"))

Input: ice cream
Formatted prompt: Human:

Tell me a short joke about ice cream

Assistant:
Output:  Sure, here's a silly one about ice cream:

Why did the ice cream cross the road?
Because it was fudge-tired!
 Sure, here's a silly one about ice cream:

Why did the ice cream cross the road?
Because it was fudge-tired!


If you wanted to add retry or fallback logic:

In [ ]:
def manual_chain_with_fallback(topic: str) -> str:
    try:
        return manual_chain(topic)
    except Exception:
        return manual_chain_anthropic(topic)

### With LCEL

Now let's take a look at how all of this work with LCEL. We'll use our chain from before (and for ease of use take in a string instead of a dict):

In [48]:
from langchain_core.runnables import RunnablePassthrough

prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")
model = ChatOpenAI(model="gpt-3.5-turbo")
output_parser = StrOutputParser()

chain = {"topic": RunnablePassthrough()} | prompt | model | output_parser

In [49]:
# Simple invocation
chain.invoke("ice cream")

'Why did the ice cream go to therapy?\n\nBecause it had a rocky road!'

In [50]:
# Streaming
for chunk in chain.stream("ice cream"):
    print(chunk, end="", flush=True)

Why did the ice cream go to the dentist?

Because it had a sprinkle-tingling sensation!

In [51]:
# Parallelized batch execution
chain.batch(["ice cream", "spaghetti", "dumplings"])

['Why did the ice cream go to the party?\n\nBecause it was looking to get "scoop" of the fun!',
 'Why did the spaghetti go to the spa?\n\nBecause it needed to relax and pasta time!',
 'Why did the dumpling go to therapy?\n\nBecause it had too many "wrapper" issues!']

In [ ]:
# Async
# chain.ainvoke("ice cream)

In [52]:
# Using completion model
from langchain.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-instruct")
llm_chain = {"topic": RunnablePassthrough()} | prompt | llm | output_parser
llm_chain.invoke("ice cream")

'\n\nWhy did the ice cream go to therapy?\n\nBecause it was feeling a little melon-choly.'

If we wanted, we could even make the choice of chat model or llm runtime configurable

In [53]:
from langchain_core.runnables import ConfigurableField

configurable_model = model.configurable_alternatives(
    ConfigurableField(id="model"), default_key="chat_openai", openai=llm
)
configurable_chain = {"topic": RunnablePassthrough()} | prompt | llm | output_parser
configurable_chain.invoke("ice cream")

'\n\nWhy did the ice cream go to therapy? Because it was feeling a little rocky road.'

In [54]:
configurable_chain.invoke("ice cream", config={"configurable": {"model": "openai"}})

'\n\nWhy did the ice cream truck break down?\n\nBecause it had a meltdown.'

In [55]:
# Using anthropic
from langchain.chat_models import ChatAnthropic

anthropic = ChatAnthropic(model="claude-2")
anthropic_chain = {"topic": RunnablePassthrough()} | prompt | anthropic | output_parser
anthropic_chain.invoke("ice cream")

' Why did the ice cream cone go to the gym? To get more scoops!'

By turning on LangSmith, every step of every chain is automatically logged. We set these environment variables:

In [ ]:
import os

os.environ["LANGCHAIN_API_KEY"] = "..."
os.environ["LANGCHAIN_TRACING_V2"] = "true"

And then get a trace of every chain run: {trace}

In [ ]:
# Fallbacks

fallback_chain = chain.with_fallbacks([anthropic_chain])